In [ ]:
#LOADING AND DISPLAYING IMAGES FROM THE DATASET
import os
import cv2
import random
dataset_path = r"C:\Users\nethr\Downloads\FYP\archive" #harcoded path come to it later
image_folder_path = os.path.join(dataset_path, 'train')
print(f"Looking for images in: {image_folder_path}")

#Lists the image files
try:
    all_files = os.listdir(image_folder_path)
    image_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    if not image_files:
        print(f"Error: No image files found in {image_folder_path}. Please check the path and folder structure.")
        exit()
    print(f"Found {len(image_files)} image files.")
except FileNotFoundError:
    print(f"Error: The directory {image_folder_path} does not exist. Please check the path.")
    exit()

#Displays 10 random images
num_images_to_show = 10
random_image_files = random.sample(image_files, min(num_images_to_show, len(image_files))) 
print(f"Displaying {len(random_image_files)} random images...")

#Load and displays images
for image_name in random_image_files:
    image_path = os.path.join(image_folder_path, image_name)
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Could not load image {image_name}. Skipping.")
        continue 
    #Resizes for display
    height, width = img.shape[:2] 
    max_width = 800 
    if width > max_width:
        ratio = max_width / width  # Calculate the ratio to resize by
        new_height = int(height * ratio) # Calculate new dimensions
        img_display = cv2.resize(img, (max_width, new_height)) # Resizing the image
    else:
        img_display = img
    cv2.imshow(image_name, img_display) 

print("\nPress any key to close the image windows.")
cv2.waitKey(0)
cv2.destroyAllWindows()

Looking for images in: C:\Users\nethr\Downloads\FYP\archive\train
Found 8691 image files.
Displaying 10 random images...

Press any key to close the image windows.
